In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.shape

(891, 12)

In [4]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
train.groupby('Survived').size()

Survived
0    549
1    342
dtype: int64

In [6]:
train.value_counts('Survived', normalize=True)

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64

In [7]:
#성별 생존율, 사망률
total = train.groupby('Sex').size().to_frame('전체인원')
total


,전체인원
Sex,
female,314
male,577


In [8]:
filt = train['Survived'] == 1
survived = train[filt]
survived = survived.groupby('Sex').size().to_frame('생존인원')
survived

,생존인원
Sex,
female,233
male,109


In [9]:
gender = total.merge(survived, on='Sex')
gender

,전체인원,생존인원
Sex,,
female,314,233
male,577,109


In [10]:
gender['생존율'] = gender['생존인원']/gender['전체인원'] * 100
gender

,전체인원,생존인원,생존율
Sex,,,
female,314,233,74.203822
male,577,109,18.890815


In [11]:
total.join(survived)

,전체인원,생존인원
Sex,,
female,314,233
male,577,109


In [12]:
#티켓 클래스별 생존율 Pclass
total = train.groupby('Pclass').size().to_frame('전체인원')
total

,전체인원
Pclass,
1,216
2,184
3,491


In [13]:
filt = train['Survived'] == 1
survived = train[filt]
pclass = survived.groupby('Pclass').size().to_frame('생존인원')
pclass

,생존인원
Pclass,
1,136
2,87
3,119


In [14]:
pclass = total.merge(pclass, on='Pclass')
pclass

,전체인원,생존인원
Pclass,,
1,216,136
2,184,87
3,491,119


In [15]:
pclass['생존율'] = pclass['생존인원']/pclass['전체인원'] * 100
pclass

,전체인원,생존인원,생존율
Pclass,,,
1,216,136,62.962963
2,184,87,47.282609
3,491,119,24.236253


In [16]:
train['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [17]:
train['Age'].min(), train['Age'].max()

(np.float64(0.42), np.float64(80.0))

In [18]:
min = int(train['Age'].min())
max = int(train['Age'].max())
min, max

(0, 80)

In [19]:
ages = []
step = 20
for age in range(0, int(train['Age'].max()+1), step):
    filt = (train['Age'] >= age) & (train['Age'] < age+step)
    total = train[filt]
    count = train[filt]['Age'].count() #연령대별 전체인원

    filt = (train['Age'] >= age) & (train['Age'] < age+step) &(train['Survived']==1)
    count_survived = train[filt]['Age'].count() #연령대별 생존자수
    ages.append({'나이':f'{age}세 이상 ~ {age+step}세 미만', 
                 '전체인원':count,
                 '생존인원':count_survived})

df = pd.DataFrame(ages)
df.set_index('나이', inplace=True)
df['생존율'] = df['생존인원']/df['전체인원']
df


,전체인원,생존인원,생존율
나이,,,
0세 이상 ~ 20세 미만,164,79,0.481707
20세 이상 ~ 40세 미만,387,150,0.387597
40세 이상 ~ 60세 미만,137,54,0.394161
60세 이상 ~ 80세 미만,25,6,0.240000
80세 이상 ~ 100세 미만,1,1,1.000000


In [20]:
df['전체인원'].sum()

np.int64(714)

In [21]:
#탑승항구(Embarked)별 전체인원, 생존인원, 생존율

In [22]:
group = train.groupby(['Embarked', 'Pclass']).size().to_frame()
group

0
Embarked Pclass     
C        1        85
         2        17
         3        66
Q        1         2
         2         3
         3        72
S        1       127
         2       164
         3       353

In [23]:
etotal = train.groupby('Embarked').size().to_frame('전체인원')
etotal

,전체인원
Embarked,
C,168
Q,77
S,644


In [24]:
filt = train['Survived'] == 1
survived = train[filt]
embarked = survived.groupby('Embarked').size().to_frame('생존인원')
embarked


,생존인원
Embarked,
C,93
Q,30
S,217


In [25]:
embarked = etotal.merge(embarked, on='Embarked')
embarked

,전체인원,생존인원
Embarked,,
C,168,93
Q,77,30
S,644,217


In [26]:
embarked['생존율'] = embarked['생존인원']/embarked['전체인원']
embarked

,전체인원,생존인원,생존율
Embarked,,,
C,168,93,0.553571
Q,77,30,0.389610
S,644,217,0.336957


In [27]:
group = train.groupby(['Embarked','Pclass']).size()
c = group['C']
s = group['S']
q = group['Q']
df = pd.DataFrame([c,s,q], index=['C','S','Q'])
df.index.name = '항구'
df.columns=['1등석','2등석','3등석']
df

,1등석,2등석,3등석
항구,,,
C,85,17,66
S,127,164,353
Q,2,3,72


In [28]:
import pandas as pd

In [30]:
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
group = train.groupby(['Embarked', 'Pclass']).size()
group

Embarked  Pclass
C         1          85
          2          17
          3          66
Q         1           2
          2           3
          3          72
S         1         127
          2         164
          3         353
dtype: int64

In [37]:
df = pd.DataFrame()
for idx in group.index:
    df.loc[idx[0], idx[1]] = group.loc[idx]

df.index.name = '항구'
cols = df.columns
df.columns = [str(col)+'등석' for col in cols]
df

,1등석,2등석,3등석
항구,,,
C,85.0,17.0,66.0
Q,2.0,3.0,72.0
S,127.0,164.0,353.0
